In [3]:
import random
class barco: #representa un barco en el juego
    def __init__(self, nombre:str, tamaño:float, posicion:list, golpes):
        self.nombre= nombre #nombre de los barcos
        self.tamaño= tamaño
        self.posicion= posicion
        self.golpes=0 # para contar los golpes 
    def recibir_golpe(self,posicion:list):
        if posicion in self.posicion:  #si da una coordenada donde esta el barco, suma un golpe, si supera el tamaño del barco, hundir el barco
            if self.tamaño < self.golpes: 
                self.golpes+=1
            print(f"Golpe al barco {self.nombre}")
        else:
            print(f"Barco {self.nombre} ha sido hundido")
        
    def esta_hundido(self):
        return self.golpes>=self.tamaño #indica cuando un barco se ha hundido 
    


class Tablero:
    def __init__(self):
        self.tablero = [[0 for _ in range(10)] for _ in range(10)] #crea un tablero 10x10 dejando espacios
        self.barco ={} #diccionario de barcos en el tablero
         
    def colocar_barco(self, nombre_barco, posicion, orientacion,tamaño):
        posiciones_barcos = [] # Almacenar la posición de cada barco para el seguimiento
        fila, columna = posicion
       

        if orientacion == "h":
            # Verificar límites y disponibilidad de espacio horizontal
            if columna + tamaño > 10: #si se sale del rango 
                return False
            for i in range(tamaño):
                if self.tablero[fila][columna + i] != 0: #si ya hay un barco en esa posición
                    return False
                posiciones_barcos.append((fila, columna + i)) #se agrega a la lista los barcos colocados
            
        elif orientacion == "v":
            if fila + tamaño > 10:
                return False
            for i in range(tamaño):
                if self.tablero[fila + i][columna] != 0:
                    return False
                posiciones_barcos.append((fila, columna + i))
        
        
            
    def mostrar_tablero(self):
        for fila in self.tablero:
            print(" ".join(str(celda) for celda in fila)) #recorre el tablero convirtiendolo en una cadena y lo une con un espacio

    def recibir_ataque(self, fila, columna):
        #verifica si da al barco y lo marca como golpeado
        if self.tablero[fila][columna] == "B": 
            self.tablero[fila][columna] = "X"  # Golpe 
            print("¡Impacto!")
            return True
        elif self.tablero[fila][columna] == "X": #comprueba si ya se ha dado a esa coordenada
            print("Ya golpeaste esta posición.")
            return False
        else:
            self.tablero[fila][columna] = "O"  # Agua
            print("Has fallado.")
            return False

    def todos_barcos_hundidos(self):
        for i in self.barco.values(): #recorre todas las posiciones de todos los barcos
            if any(self.tablero[fila][columna]=="B"for fila, columna in i): #si algun barco sigue 
                return False  # Algun barco aun sigue en juego
        return True  # Todos los barcos han sido hundidos
        

In [4]:
class Jugador:
    def __init__(self, nombre, tablero):
        self.nombre = nombre #nombre del jugador
        print(f"Bienvenido {self.nombre} al juego de hundir la flota!")
        tablero=Tablero

    def atacar(oponente,fila,columna):
        resultado=oponente.tablero.recibir_ataque(fila, columna) #utiliza el metodo para recibir el ataque del oponente
        print(f"el jugador{self.nombre} ha atacado a {fila}, {columna} con resultado: {resultado}")
        return resultado

In [5]:
class Juego:
    def __init__(self, jugador1, jugador2):
        self.jugador1 = jugador1
        self.jugador2 = jugador2
        self.tablero1 = Tablero()
        self.tablero2 = Tablero()
    def colocar_barcos(self):
        barcos_1={
            "portaaviones": 5,
            "acorazado": 4,
            "crucero": 3,
            "submarino": 3,
            "destructor": 2
        }
        for nombre, tamaño in barcos_1.items: #poner posiciones aleatorias con los tamaños y nombres de los barcos 
            fila=random.randint(0,9)
            columna=random.randint(0,9)
            orientacion=random.choice(["h","v"])
            colocado=tablero.colocar_barco(nombre, (fila,columna), orientacion, tamaño)
    def realizarataque(tablero2):
        while True:
            try:
                fila=int(input("Ingrese una fila a la que atacar: "))
                columna=int(input("Ingrese una columna a la que atacar: "))
                if 0<=fila<=9 and 0<=columna<=9:
                    break
                else:
                    print("Coordenadas fuera del rango.")
            except ValueError:
                    print("Debe ingresar unas coordenadas validas.")
            return tablero2.recibir_ataque(fila, columna)
    


        



     
    

In [6]:
import random

class Juego:
    def __init__(self, jugador1, jugador2):
        self.jugador1 = jugador1
        self.jugador2 = jugador2
        self.tablero1 = Tablero()
        self.tablero2 = Tablero()

    def configurar_barcos(self):
        # Fase de colocación de barcos para cada jugador
        print(f"{self.jugador1}, coloca tus barcos.")
        self.colocar_barcos_automatico(self.tablero1)
        print(f"{self.jugador2}, coloca tus barcos.")
        self.colocar_barcos_automatico(self.tablero2)

    def colocar_barcos_automatico(self, tablero):
        # Definir los barcos con sus tamaños y nombres
        barcos = {"Portaaviones": 5, "Acorazado": 4, "Submarino": 3, "Destructor": 3, "Patrullero": 2}
        
        for nombre, tamaño in barcos.items():
            colocado = False
            while not colocado:
                fila = random.randint(0, 9)
                columna = random.randint(0, 9)
                orientacion = random.choice(["h", "v"])
                colocado = tablero.colocar_barco(nombre, (fila, columna), orientacion, tamaño)
            print(f"{nombre} colocado en el tablero.")

    def jugar(self):
        self.configurar_barcos()
        turno = 1  # 1 para jugador 1, 2 para jugador 2

        while True:
            if turno == 1:
                print(f"Turno de {self.jugador1}")
                ataque_exitoso = self.realizar_ataque(self.tablero2)
                if self.tablero2.todos_barcos_hundidos():
                    print(f"¡{self.jugador1} ha ganado!")
                    break
                if not ataque_exitoso:
                    turno = 2
            else:
                print(f"Turno de {self.jugador2}")
                ataque_exitoso = self.realizar_ataque(self.tablero1)
                if self.tablero1.todos_barcos_hundidos():
                    print(f"¡{self.jugador2} ha ganado!")
                    break
                if not ataque_exitoso:
                    turno = 1

    def realizar_ataque(self, tablero_enemigo):
        while True:
            try:
                fila = int(input("Ingrese la fila del ataque (0-9): "))
                columna = int(input("Ingrese la columna del ataque (0-9): "))
                if 0 <= fila < 10 and 0 <= columna < 10:
                    break
                else:
                    print("Coordenadas fuera de rango. Intente nuevamente.")
            except ValueError:
                print("Por favor, ingrese números válidos.")

        return tablero_enemigo.recibir_ataque(fila, columna)
    
jugador1 = "Jugador 1"
jugador2 = "Jugador 2"
juego = Juego(jugador1, jugador2)
juego.jugar()

    


Jugador 1, coloca tus barcos.
